In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
import tensorflow as tf
import numpy as np
import gym
from tqdm import tqdm, trange
import os, sys
sys.path.append(os.getcwd())

Populating the interactive namespace from numpy and matplotlib


In [3]:
class TicTacToe():
    def __init__(self, state=None):
        self.reset()
        if state is not None:
            self.state = state
        
    def reset(self):
        self.done = False
        self.state = [0]*9
        self.turn = 1
        return self.state
    
    class observation_space():
        shape = (11,)

    class action_space():
        n = 9

    def render(self):
        print(self.state)
        # print(np.array(self.state[0:9]).reshape(3,3))
    
    def value(self, s):
        ret = 0
        for turn in [-1, 1]:
            for i in range(3):
                if all([x==turn for x in s[3*i:3*i+3]]):
                    ret = turn
                if all([x==turn for x in [s[i], s[3+i], s[6+i]]]):
                    ret = turn
                if all([x==turn for x in [s[0], s[4], s[8]]]):
                    ret = turn
                if all([x==turn for x in [s[2], s[4], s[6]]]):
                    ret = turn
                # NOTE: this is not the value, the state may be won
                return ret*s[-1]


    def step(self, act):
        rew, self.state = self.dynamics(self.state, act)
        if rew != 0:
          self.done = True
        if np.all(np.array(self.state[0:9]) != 0):
          self.done = True
        return self.state, rew, self.done, None

In [4]:
env = TicTacToe()

In [5]:
env.render()

[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
from muzero.model import MuModel
m = MuModel(env.observation_space.shape, env.action_space.n, s_dim=64, K=3, lr=0.001)
print(env.observation_space.shape, env.action_space.n)

from muzero.game import Game, ReplayBuffer
from muzero.mcts import naive_search, mcts_search
replay_buffer = ReplayBuffer(50, 128, m.K)
rews = []

(11,) 9
